In [1]:
import tmscoring

de_novo_filepath = '../../datasets/alphafold_test_output/antibody_de_novo.pdb'
original_filepath = '../../datasets/alphafold_test_output/antibody_real.pdb'

alignment = tmscoring.TMscoring(de_novo_filepath, original_filepath)
alignment.optimise()
print(alignment.tmscore(**alignment.get_current_values()))
print(alignment.tmscore_samples(**alignment.get_current_values()))
print(alignment.rmsd(**alignment.get_current_values()))

0.9961657282211568
[0.9065317  0.97646536 0.99066769 0.99943629 0.99890903 0.99974941
 0.99919408 0.9990635  0.99304587 0.99879329 0.99946023 0.99920663
 0.99860702 0.99953093 0.99966989 0.9994749  0.99404764 0.99794738
 0.99951838 0.99976513 0.9989943  0.99973341 0.99947068 0.99996409
 0.99173974 0.97946622 0.97217669 0.99324986 0.99753606 0.99450997
 0.995163   0.99690499 0.99958994 0.99958745 0.9981017  0.99974747
 0.99965754 0.99974237 0.9999491  0.99945208 0.98905179 0.94714656
 0.98124665 0.99985969 0.99957074 0.99821128 0.998696   0.99876352
 0.99969081 0.99919732 0.99982679 0.99936568 0.99979322 0.99961682
 0.99966988 0.99954144 0.999707   0.99956972 0.99842408 0.99288001
 0.99014968 0.99701254 0.99750572 0.9967157  0.99644031 0.99818758
 0.99972847 0.99978298 0.99955749 0.99934215 0.99991833 0.99986573
 0.99974039 0.9997296  0.99972779 0.99965105 0.99949743 0.99973922
 0.99961838 0.99994651 0.99996183 0.99960863 0.99936582 0.99960562
 0.99987396 0.99950064 0.99793754 0.9968109